# CSE152B: Homework 2
## Computing Resources
Please read the README file of this repository for the instructions
## Instructions
1. Attempt all questions.
2. Please comment all your code adequately.
3. Include all relevant information such as text answers, output images in notebook.
4. **Academic integrity:** The homework must be completed individually.

5. **Submission instructions:**  
 (a) Submit the notebook and its PDF version on Gradescope.  
 (b) Rename your submission files as Lastname_Firstname.ipynb and Lastname_Firstname.pdf.  
 (c) Correctly select pages for each answer on Gradescope to allow proper grading.

6. **Due date:** Assignments are due Thu, May 21, at 4pm. 

Git clone the repo of HW2:
```
git clone https://github.com/Jerrypiglet/cse152b_hw2-release.git
cd cse152b_hw2-release
```

## Q1: Using SphereFace [3] for Face Verification

1. In the first section, we will test a pretrained model of SphereFace on LFW [4] dataset. The LFW dataset is on `/datasets/cse152-252-sp20-public/hw2_data/lfw`. The dataset contains 6000 pairs of human face images with ground truth labels for whether they are from the same identity.
2. The PyTorch code of SphereFace is located in `./sphereFace`,  which is modified based on the open source code from `https://github.com/clcarwin/sphereface_pytorch`. 
3. Run the following commands and report the accuracy of SphereFace on LFW verification. **(5 points)**
```
cd sphereFace
tar -zxf model.tar.gz
python lfw_eval.py --model ./model/sphere20a_20171020.pth --net faceNet --lfw /datasets/cse152-252-sp20-public/hw2_data/lfw/
```

``Answer here.``  
LFWACC=0.9918 std=0.0051 thd=0.3095

4. Explain briefly how the following steps are performed when evaluating on LFW dataset: 
    1. Given the features extracted from the network, what is the metric used to measure the distance between two faces? (`lfw_eval.py`: Line 135) **(5 points)**  
    Here we are using the Angular Softmax that generates a softmax loss function that is normalized. Therefore the usual exponent $W^T x$ (assuming the bias is 0), becomes $||x||cos(m \theta)$. In other words it is using an angular margin between classes.     
    2. How is the threshold set to determine whether two faces are from the same identity? How is the accuracy computed? (`lfw_eval.py`: Line 141 to 148) **(10 points)**.  
    First both images are concatenated together via np.vstack and given to the network together. The output is a cosine distance between the two faces as explained in part 1 above.
    From the output predictions from the network, we find the best threshold from the list of (-1, 1, 0.005) by running all the predictions distances against these, and finding which threshold yields the highest accuracy rates. After finding this nice threshold, we run all the predictions for this kfold iteration on that chosen threshold and get an accuracy for this kfold iteration. 
    Accuracy is computed how it normally is : The number of true positives over the number of total predictions.   
    We use the K-fold evaluation of validation in order to get the accuracy after training. We divide the training data into K folds, and alternate where each fold in the K folds act as the validation set each iteration. After repeating the above to all the kfolds, we get the average accuracy overall the kfold iterations.  
    

5. An important step before face recognition is face alignment, in which we warp and crop the image based on the location of facial landmarks.
    1. Briefly describe how we warp and crop the image. (`lfw_eval.py`: Line 11-26) **(5 points)**   
    We have a preset array of 2d reference points of a face that we want the facial landmark features to be in at the end called ref_pts and a designated crop size that we want the image to be in. The given parameter src_pts (all pre processed in data/lfw_landmark.txt) tells us where these "facial landmarks" for each face are. We then calculate the affine transformation for the given src_points to line up with the ref_pts as well as possible (also while cropping the face into the specified size) using the function get_similarity_transform_for_PIL. We then return this new augmented face.  
    
    2. Instead of doing face alignment, crop an image patch of height 112 pixels and width 96 pixels at the center of the image. Report the accuracy.  **(10 points)**  
    LFWACC=0.8748 std=0.0221 thd=0.2300
    An observation that I would like to point out is that making this a simpler pre processing step, has decreased the training time by many times, but has lead to horrible accuracies due to the inefficient pre procressing augmentation. 

## Q2: Using MTCNN [5] for Detecting Face Landmarks

1. Instead of using provided facial landmarks, we will now use MTCNN [5] for detecting them. The code is located at `./mtcnn`. Run the following commands to generate the facial landmarks. Include two example outputs in your report. **(5 points)**
```
cd MTCNN 
python lfw_landmark.py --lfw /datasets/cse152-252-sp20-public/hw2_data/lfw/
```

Zurab_Tsereteli/Zurab_Tsereteli_0001.jpg        100.228 114.919 143.161 110.304 120.893 139.357 108.622      159.118 148.102 155.127
Zydrunas_Ilgauskas/Zydrunas_Ilgauskas_0001.jpg  109.997 115.973 144.402 112.668 133.919 129.575 119.971      159.418 144.310 156.468

2. Go to the `sphereFace` directory, run the following commands by setting flag `alignmentMode` to be 2. Report the error using the predicted facial landmarks. **(5 points)**
```
cd sphereFace
python lfw_eval.py --model ./model/sphere20a_20171020.pth --net faceNet --lfw /datasets/cse152-252-sp20-public/hw2_data/lfw/ --alignmentMode 2
```

LFWACC=0.9845 std=0.0057 thd=0.2995

3. Next, answer the following questions:
    1. Is the result better than using the landmarks provided in the previous question? If not, how can you improve performance? **(5 points)**  
    
    One of the problems is that if the image detects more than one face in the image, it just chooses the largest face in the array of landmarks. If we can hand crop the faces so that there are only one face per image, this can dramatically increase the 'clean-ness' of the dataset and yield higher accuracy rates. 
    2.  What are the steps adopted by the method to achieve real-time speed? **(5 points)**  
    Uses Cuda network inferencing for one thing. In addition, they downscale the faces in lines 
    3. Briefly describe how non-maximal suppression (NMS) is implemented in this method. (`src/box_utils.py`: Line 5-68) **(5 points)**  
    NMS attempts to limit the amount of redundant bounding boxes over the same object in the image.  
    It will start from the largest box, and calculate the intersection between itself and the rest (smaller) boxes. It will union or take the minimum box (depending on the option) and delete boxes in which the overlap is so large with the current box that they are not taken into consideration (This threshold starts at 0.5). 

## Q3: Training CosFace [7] on CASIA Dataset [6]

In this section, you are required to implement CosFace based on the code you use in the previous section, train it on CASIA dataset and test on LFW dataset. 
1. You will now train CosFace [7] on the CASIA dataset [6] and test on the LFW dataset [4]. In this section, the skeleton code for training is given. CASIA-Webface dataset can be found at `/datasets/cse152-252-sp20-public/hw2_data/CASIA-WebFace`.  

Go to directory `./cosFace` and open `faceNet.py`. Under class `CustomLienar` implement function $\psi(\theta_{y_i}, i)  = \cos(\theta_{y_i}, i) - m$ which is the cosine distance with margin $m$.
Under class `CustomLoss`, implement the loss function with the returned $\psi(\theta_{y_i}, i)$ and $\cos(\theta_{j}, i)$. You may check (and duly cite) any open source implementation for hints on improving the performance. **(15 points)**

``Copy your implementation of CustomLinear and CustomLoss here``

2. The original architecture of FaceNet in ``cosFace/FaceNet.py`` is a 20-layer residual network as described in Table 2 of [3], but without batch normalization. Now add batch normalization after every convolutional and fully connected layer. Train the new network on CASIA dataset and test on LFW dataset. You are free to change any hyper parameters but report the hyper parameters that you think might influence the performance. Following is a demonstration of a residual block with 128 filters and kernel size $3\times3$:
\begin{align}
y &=& \mathtt{CONV}_{3\times3, 128}(x) \\
y &=& \mathtt{BatchNorm}(y) \\
y &=& \mathtt{PReLU}(y) \\
y &=& \mathtt{CONV}_{3\times3, 128}(y) \\
y &=& \mathtt{BatchNorm}(y) \\
y &=& \mathtt{PReLU}(y) \\
\mathtt{OUT} &=& x + y
\end{align}
    1. Draw the training curves for accuracy and loss on CASIA and compare to the curve without batch normalization. **(10 points)**
    2. Report accuracy on the LFW dataset, evaluated using `lfw_eval.py`. **(10 points)**
    3. Do you achieve better performance on LFW? If yes, explain how batch normalization helps. If not, try to explain why the results are worse.  **(10 points)**

``Answer here.``

3. If you achieve better performance compared to SphereFace in Q1, well done! Can you provide a reason? If you do not outperform SphereFace, can you provide a cause? **(10 points)**

``Answer here.``

4. Plot the tSNE visualization of the CosFace embedding for the same identities from the CASIA dataset used to visualize the SphereFace embedding in the previous question. **(10 points)**

``Answer here.``

## Q4: Human Pose Estimation with Convolutional Pose Machines (CPM) [1]

In this question, you will be given code adapted from [PyTorch implementation of one of the CPM model [1]](https://github.com/Hzzone/pytorch-openpose), which is an follow-up work of the original Convolution Pose Machine [2], and shares the Part Confidence Maps estimation module with [2]. In this question you will be given a trained CPM model and gain insights about the model design and outputs.

- ``cd pytorch-openpose``
- Set up the environment following the instructions at ``pytorch-openpose/README.md`` (Getting Started - Install Requriements)
- Download the ``body_pose_model.pth`` from [Dropbox](https://www.dropbox.com/sh/7xbup2qsn7vvjxo/AABWFksdlgOMXR_r5v3RwKRYa?dl=0) and place under ``pytorch-openpose/model``

1. Run the inference code on the given image.

In [ ]:
import cv2
import matplotlib.pyplot as plt
import copy
import numpy as np

from src import model
from src import util
from src.body import Body
from src.hand import Hand

body_estimation = Body('model/body_pose_model.pth')

test_image = 'images/demo.jpg'
oriImg = cv2.imread(test_image)  # B,G,R order
candidate, subset, heatmap_list, heatmap_list_converted_list = body_estimation(oriImg)
heatmap_0 = heatmap_list[0] 
canvas = copy.deepcopy(oriImg)
canvas = util.draw_bodypose(canvas, candidate, subset)

plt.figure(figsize=(10, 10))
plt.imshow(canvas[:, :, 
                  [2, 1, 0]])
# plt.axis('off')
plt.show()

(a) Visualize the output keypoint detection result. **(5 points)**

``Paste the output figure here``

(b) What is the ratio of size $\lambda=H/H'$ between the input image **im** (Line 57 of ``body.py``) 
of shape [1, 3, H, W] and the output heatmap **heatmap_0** of shape [1, D, H', W']? **(5 points)**

``Answer the question here``

(c) What module in the model is reponsible for this scaling? **(5 points)**

``Answer the question here``

2. ``heatmap_list_converted_list[0]`` is a list of heatmaps from all 6 layers: [out1_2, out2_2, out3_2, out4_2, out5_2, out6_2], where the output of each layer is of shape [H, W, D].

This snippet gives the function to visualize the $d_{th}$ feature map from the layer **layer_idx**

In [ ]:
layer_idx = -1

for d in range(heatmap_list_converted_list[0][layer_idx].shape[2]):
    layer_idx = 0
    heatmap = heatmap_list_converted_list[0][layer_idx][:, :, d]
    util.overlay_heatmap(canvas, heatmap)

(a) What is D? And given D can you tell what is the number of keypoints that the model is trying to estimate? **(5 points)**

``Answer here``

(b) Visualize and compare the heatmap from **layer_idx=0** and **layer_idx=5** for the keypoint **d=2**. **(5 points)**

``Paste and compare the visualizations here``

(c) Visualize and compare the heatmap from **layer_idx=0** and **layer_idx=5** for the keypoint **d=3**. **(5 points)**

``Paste and compare the visualizations here``

(d) State and justify the shared difference in these comparisons? You may get some hint from the model design. **(5 points)**

``State and justify the difference here``

3. Set **multi_scale=False** in Line 38 of ``body.py`` and re-run the code from Q4(1). 

    (a) Visualize the output below. **(5 points)**
    
    **NOTE: you might need to restart the kernel each time you change the ``multi_scale`` flag. This may be required to re-initialize the model. [Kernel]-->[Restart]**

``Paste figure by re-running (a) with multi_scale=False here``

(b) Do the same comparison with additional images of ``demo_coco.jpg``, ``demo_coco2.jpg``, ``demo_beach.jpg``, ``demo_ucsd.jpg``, ``demo_ucsd2.jpg``.  **(5 points)**

``For each of the additional images, paste two figures with multi_scale=True and multi_scale=False here``

(c) What shared difference can you spot in most of the comparisons? Justify the difference. **(5 points)**

``Report the difference and jsutify here``

# Reference
[1] Cao, Zhe. et al. "Realtime multi-person 2d pose estimation using part affinity fields." Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition. 2017.

[2] Wei, Shih-En. et al. "Convolutional pose machines." Proceedings of the IEEE conference on Computer Vision and Pattern Recognition. 2016.

[3] Liu, Weiyang. et al. "SphereFace: Deep Hypersphere Embedding for Face Recognition." arXiv:1704.08063.

[4] Huang, Gary. et al. "Labeled faces in the wild:  Adatabase for studying face recognition in unconstrained environments." Technical Report 07-49, Universityof Massachusetts, Amherst, October 2007.

[5] Zhang, Kaipeng. et al. "Joint face detection and alignment usingmultitask cascaded convolutional networks." IEEE Signal Processing Letters, 23(10):1499–1503, 2016.

[6] Yi, Dong. et al.  Learning face representation from scratch. arXiv:1411.7923.

[7] Wang, Hao. et al.Cosface: Large margin cosine loss for deep face recognition. In Proceedings of the IEEE Conference onComputer Vision and Pattern Recognition, pages 5265–5274, 2018.